In [1]:
import os
import random
import yaml
import pickle
import sys
sys.path.append('./tsfm_public')
# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
)
import numpy as np
import pandas as pd
import torch
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index
from read_data import loadData
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

c:\Users\02213R744\anaconda3\envs\patch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_train, testDatasets, expectedRulDatasets = loadData()

In [3]:
drop_col = ['setting_1', 'setting_2', 'setting_3'] 
data_clean = [data.drop(columns = drop_col) for data in data_train]

In [4]:
df_all = pd.DataFrame()
for i in range(4):
    df_all =pd.concat([df_all, data_clean[i]])

In [5]:
for engine in list(set(df_all['engine'])):
    max_cycle = df_all[df_all['engine'] == engine]['cycle'].max() 
    condition = (df_all['engine'] == engine) & (df_all['cycle'] > max_cycle - 25)
    df_all.loc[condition, 'label'] = 1

In [6]:
df_all = df_all.fillna(0)

In [7]:
df_all['label'].value_counts()

label
0.0    152974
1.0      7385
Name: count, dtype: int64

In [8]:
df_engine_1 = df_all.drop(columns=['index'], errors='ignore')

In [9]:
# df_engine_1 = pd.DataFrame()
# for i in range(len(data_clean)):
#     df = data_clean[i][data_clean[i]['engine'] == 1].copy()  # Make a copy of the DataFrame
#     max_cycle = df['cycle'].max()
#     for j in range(df.shape[0]):
#         if df.loc[j, 'cycle'] > max_cycle - 25:
#             df.loc[j, 'label'] = 1
#         else:
#             df.loc[j, 'label'] = 0
#     df_engine_1 = pd.concat([df_engine_1, df], ignore_index=True)

In [10]:
correlation_matrix = df_engine_1.drop(columns=['engine']).corr()

In [11]:
high_corr_columns = correlation_matrix[abs(correlation_matrix['label'] )> 0.03].index.tolist()
# high_corr_colm = correlation_matrix[abs(correlation_matrix['label'] )> 0.03].T.columns

df_sample = df_engine_1[high_corr_columns]

In [25]:
df_sample.columns

Index(['cycle', 'Fan_inlet_temperature_R', 'Fan_inlet_Pressure_psia',
       'bypass_duct_pressure_psia', 'HPC_outlet_pressure_psia',
       'Ratio_of_fuel_flow_to_Ps30_pps_psia', 'Corrected_core_speed_rpm',
       'High_pressure_turbines_Cool_air_flow',
       'Low_pressure_turbines_Cool_air_flow', 'index'],
      dtype='object')

In [12]:
df_sample = df_sample.drop(columns=['label'])

In [13]:
df_sample.shape

(160359, 9)

In [14]:
# correlation_matrix = df_engine_1.drop(columns=['engine']).corr()
# high_corr_colm = correlation_matrix[abs(correlation_matrix['label'] )> 0.03].T.columns
# df_engine_1_sample_1 = df_engine_1[high_corr_colm]
# df_engine_1_sample = df_engine_1_sample_1.drop(columns=['label'])

In [15]:
target_col = None 

In [16]:
with open("./config.yaml", "r") as file:
    configs = yaml.safe_load(file)

num_workers = configs['num_workers'] # Reduce this if you have low number of CPU cores
batch_size = configs['batch_size']  # Reduce if not enough GPU memory available
context_length = configs['context_length'] 
forecast_horizon = configs['forecast_horizon']  # 8 hours 
patch_length = configs['patch_length'] 
# target_col = configs['target_columns']

timestamp_column = "cycle"

In [17]:
df_sample.loc[:, 'index'] = df_sample.index
data = df_sample.copy()

In [18]:
data.head(3)

,cycle,Fan_inlet_temperature_R,Fan_inlet_Pressure_psia,bypass_duct_pressure_psia,HPC_outlet_pressure_psia,Ratio_of_fuel_flow_to_Ps30_pps_psia,Corrected_core_speed_rpm,High_pressure_turbines_Cool_air_flow,Low_pressure_turbines_Cool_air_flow,index
0,1,518.67,14.62,21.61,554.36,521.66,8138.62,39.06,23.4190,0
1,2,518.67,14.62,21.61,553.75,522.28,8131.49,39.00,23.4236,1
2,3,518.67,14.62,21.61,554.26,522.42,8133.23,38.95,23.3442,2


In [19]:
timestamp_column = 'index'

In [20]:
id_columns = []
forecast_columns = [col for col in data.columns if col not in timestamp_column]
train_start_index = None  # None indicates beginning of dataset
train_end_index = int(len(data)*configs['train_data_split']) 
# we shift the start of the validation/test period back by context length so that the first validation/test timestamp is immediately following the training data
valid_start_index = int(len(data)*configs['train_data_split']) - context_length 
valid_end_index = int(len(data)*configs['train_data_split']) + int(len(data)*configs['valid_data_split']) 
test_start_index = int(len(data)*configs['train_data_split']) + int(len(data)*configs['valid_data_split']) - context_length 
test_end_index = len(data) 

train_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=train_start_index,
    end_index=train_end_index,
)
valid_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=valid_start_index,
    end_index=valid_end_index,
)
test_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=test_start_index,
    end_index=test_end_index,
)
tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    scaling=True,
)

tsp.train(train_data)

train_dataset = ForecastDFDataset(
    tsp.preprocess(train_data),
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
valid_dataset = ForecastDFDataset(
    tsp.preprocess(valid_data),
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
test_dataset = ForecastDFDataset(
    tsp.preprocess(test_data),
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)

In [21]:
with open("../model/patch_model/timeseriesprocessor.pkl", "wb") as file:
    pickle.dump(tsp, file)

In [22]:
config = PatchTSMixerConfig(
        context_length=context_length,
        prediction_length=forecast_horizon,
        patch_length=patch_length,
        num_input_channels=len(forecast_columns),
        patch_stride=patch_length,
        d_model=48,
        num_layers=3,
        expansion_factor=3,
        dropout=0.5,
        head_dropout=0.7,
        mode="common_channel", # change it `mix_channel` if we need to explicitly model channel correlations
        scaling="std",
    )
model = PatchTSMixerForPrediction(config=config)
train_args = TrainingArguments(
    output_dir="./output/",
    overwrite_output_dir=True,
    learning_rate=configs['learning_rate'],
    num_train_epochs=configs['epochs'],
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=num_workers,
    report_to="tensorboard",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="./logs/",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
)
# Create a new early stopping callback with faster convergence properties
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
)

In [23]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
)
print("\n\nDoing training")
trainer.train()
trainer.evaluate(test_dataset)
save_dir = configs['patch_foundation_model_path']
os.makedirs(save_dir, exist_ok=True)
trainer.save_model(save_dir)
print(f'Saved the foundation model at {configs["patch_foundation_model_path"]}')

c:\Users\02213R744\anaconda3\envs\patch_env\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(




Doing training


 10%|█         | 60119/601190 [15:53<2:04:59, 72.15it/s]

{'loss': 0.4861, 'learning_rate': 9e-05, 'epoch': 1.0}


                                                        
 10%|█         | 60119/601190 [16:57<2:04:59, 72.15it/s]

...

{'loss': 0.4701, 'learning_rate': 1e-05, 'epoch': 9.0}


                                                         
 90%|█████████ | 541071/601190 [2:28:34<16:30, 60.70it/s]


{'eval_loss': 0.7450457811355591, 'eval_runtime': 61.4648, 'eval_samples_per_second': 260.767, 'eval_steps_per_second': 130.384, 'epoch': 9.0}
{'train_runtime': 8914.2347, 'train_samples_per_second': 134.883, 'train_steps_per_second': 67.442, 'train_loss': 0.47339749904009826, 'epoch': 9.0}


100%|██████████| 12024/12024 [00:53<00:00, 226.18it/s]

Saved the foundation model at ../model/patch_model
